In [ ]:
from tf.app import use
A = use("etcbc/bhsa", hoist=globals())

In [ ]:
from pprint import pprint
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

In [ ]:
studied_books = ["Genesis", "Esther", "Joshua", "Psalms", "Job", "Jonah", "Song_of_songs"]

In [ ]:
{T.sectionFromNode(book)[0] for book in F.otype.s("book")}

In [ ]:
{F.vt.v(word) for word in F.otype.s("word")}

In [ ]:
verbal_tenses = ['impf', 'impv', 'infa', 'infc', 'perf', 'ptca', 'ptcp', 'wayq']

In [ ]:
column_vt = ['NA', 'impf', 'impv', 'infa', 'infc', 'perf', 'ptca', 'ptcp', 'wayq']

In [ ]:
clause_dict = {}

def get_genre(book_name):
    book_genre = {
        "Genesis": "prose", 
        "Esther": "prose", 
        "Joshua": "prose", 
        "Psalms": "poetry", 
        "Job": "poetry", 
        "Jonah": "indef", 
        "Song_of_songs": "indef",}
    return book_genre[book_name]

def get_clause_length(clause_id):
    words = L.d(clause_id, "word")
    return len(words)  

def get_verbal_tense(clause_id):
    vt = "NA"
    words = L.d(clause_id, "word")
    for word in words:
        if F.vt.v(word) in verbal_tenses:
            vt = F.vt.v(word)
    return vt      

def get_average_phrase_length(clause_id):
    phrase_lengths = []
    phrases = L.d(clause_id, "phrase")
    for phrase_id in phrases:
        words = L.d(phrase_id, "word")
        phrase_lengths.append(len(words))
    average_phrase_length = sum(phrase_lengths) / len(phrase_lengths)
    return average_phrase_length

for clause in F.otype.s("clause"):
    book, chapter, verse = T.sectionFromNode(clause)
    if book in studied_books: 
        clause_length = get_clause_length(clause)
        av_len_phr = get_average_phrase_length(clause)
        genre = get_genre(book)
        verbal_tense = get_verbal_tense(clause)
        vt_values = [0 if verbal_tense != tense else 1 for tense in column_vt]
        clause_info = [clause, book, chapter, verse, clause_length, av_len_phr, genre] + vt_values
        clause_dict[clause] = clause_info
   

In [ ]:
clause_df = pd.DataFrame(clause_dict).T

In [ ]:
clause_df.columns = ["clause", "book", "chapter", "verse", "clause_length", "av_len_phr", "genre", 'NA', 'impf', 'impv', 'infa', 'infc', 'perf', 'ptca', 'ptcp', 'wayq']

In [ ]:
clause_df.head()

In [ ]:
set(clause_df.genre)

In [ ]:
indef_books = clause_df[clause_df.genre == "indef"]

In [ ]:
indef_books.head()

In [ ]:
training_set = clause_df[clause_df.genre != "indef"]

training_set_2 = training_set.drop(["clause", "book", "chapter", "verse"], axis = 1)

y = training_set_2.pop("genre")

print(y)

In [ ]:
training_set_2

In [ ]:
clf = RandomForestClassifier(max_depth=4, random_state=0)
clf.fit(training_set_2, y)

In [ ]:
importances = clf.feature_importances_
print(importances)

In [ ]:
indef_books_2 = indef_books.drop(["clause", "book", "chapter", "verse", "genre"], axis = 1)

In [ ]:
indef_books_2

In [ ]:
prediction_genre = clf.predict(indef_books_2)

In [ ]:
book_list = list(indef_books.book)

In [ ]:
jonah_poetry = 0
jonah_prose = 0
song_poetry = 0
song_prose = 0

for pred, book in zip(prediction_genre, book_list):
    if pred == "poetry" and book == "Jonah":
        jonah_poetry += 1
    elif pred == "poetry" and book == "Song_of_songs":
        song_poetry += 1
    elif pred == "prose" and book == "Jonah":
        jonah_prose += 1
    elif pred == "prose" and book == "Song_of_songs":
        song_prose += 1

In [ ]:
print(jonah_poetry, jonah_prose, song_poetry, song_prose)